In [2]:
import torchvision
from torchvision import transforms

# Trasformazioni per le immagini (normalizzazione, ecc.)
transform = transforms.Compose([transforms.ToTensor()])

# Dataset di training
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Dataset di test
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:21<00:00, 7787162.55it/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# CIFAR-10 class names
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

# Load the saved model
model = CNN()
model.load_state_dict(torch.load('cnn_cifar10.pth'))
model.eval()

# Preprocess the custom image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    img = Image.open(image_path)
    img = transform(img)
    img = img.unsqueeze(0)  # Add batch dimension
    return img

# Make predictions with class probabilities (confidence scores)
def predict_confidence(model, image_path):
    img = preprocess_image(image_path)
    
    # Disable gradients for inference
    with torch.no_grad():
        outputs = model(img)
    
    # Apply softmax to get probabilities
    probabilities = torch.softmax(outputs, dim=1)
    
    # Convert the tensor to a Python list and get probabilities for each class
    probabilities_list = probabilities.squeeze().tolist()
    
    # Print the predicted probabilities for each class
    for i, class_name in enumerate(classes):
        print(f'{class_name}: {probabilities_list[i] * 100:.2f}%')

    # Get the predicted class (with the highest probability)
    predicted_class_idx = torch.argmax(probabilities).item()
    predicted_class = classes[predicted_class_idx]
    print(f'Predicted Class: {predicted_class}')

    return probabilities_list

# Example usage: Predict with confidence scores for a custom image
image_path = 'camion.jpg'
predict_confidence(model, image_path)


plane: 0.00%
car: 0.00%
bird: 0.00%
cat: 0.00%
deer: 0.00%
dog: 0.00%
frog: 0.00%
horse: 0.00%
ship: 14.88%
truck: 85.12%
Predicted Class: truck


C:\Users\aless\AppData\Local\Temp\ipykernel_26516\1039802102.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('cnn_cifar10.pth'))


[5.174997319556951e-09,
 6.89886758831193e-11,
 1.815909089764034e-09,
 4.053510305190855e-10,
 2.1848026859894176e-11,
 6.60582422096212e-11,
 1.7080781233858033e-11,
 1.2656365733221264e-08,
 0.14879924058914185,
 0.8512007594108582]